In [1]:
import pandas as pd, glob, calendar
from dbfread import DBF

In [6]:
css_folder = r"Z:\PhD_Datasets&Analysis\Info_Inputs\Streamflow_Stations\Climate_Sensitive_Stations-GRDC"

filtered_css = pd.read_csv(css_folder + "//CSS_FINAL_SELECTION-MERGE_WITH_ALL.csv") # Read the CSV file with all CSS stations
filtered_css

,station_name,station_no,station_id,station_latitude,station_longitude,station_status,river_name,station_elevation,CATCHMENT_SIZE,NAT_STA_ID,GRDCCOUNTRY,Continent,has_monthly_k,has_daily_k,monthly_k_recession,daily_k_recession,Next_Downstream_Station,CATCHMENT_SIZE2,Priority
0,ABERCROMBIE,5204121,1075206,-33.954800,149.325200,Active,ABERCROMBIE RIVER,426.55,"2631,00 km²",412028,AU - AUSTRALIA,Oceania,Yes,No,0.420650,NaN,5204302,2631.00,432
1,ABOVE BOULEAU CREEK,4215070,1069050,50.212223,-119.538610,Active,WHITEMAN CREEK,630.00,"112,00 km²",08NM174,CA - CANADA,North America,Yes,Yes,0.350868,0.910436,4215103,112.00,773
2,"ABOVE CHENEY RESERVOIRE, KS",4125110,1069620,37.863600,-98.014800,Active,NORTH FORK NINNESCAH RIVER,443.80,"2038,30 km²",07144780,US - UNITED STATES,North America,Yes,Yes,0.417039,0.541509,4125801,2038.30,478
3,ABOVE FORT MCPHERSON,4208040,1068040,67.248886,-134.883060,Active,PEEL RIVER,NaN,"70600,00 km²",10MC002,CA - CANADA,North America,Yes,Yes,0.625278,0.942386,NaN,70600.00,59
4,"ABOVE HAMBURG, IA",4122160,1068596,40.632500,-95.625800,Active,NISHNABOTNA RIVER,272.54,"7267,50 km²",06810000,US - UNITED STATES,North America,Yes,Yes,0.549697,0.912838,4122903,7267.50,264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,"WOODFORDS, CA (WEST FORK)",4118100,1068421,38.769600,-119.833800,Active,CARSON RIVER,1753.97,"169,38 km²",10310000,US - UNITED STATES,North America,Yes,Yes,0.593689,0.900646,NaN,169.38,732
805,XAMBIOA,3649418,1071023,-6.409700,-48.542200,Active,RIO ARAGUAIA,118.00,"364500,00 km²",28300000,BR - BRAZIL,South America,Yes,Yes,0.627802,0.974853,3649419,364500.00,20
806,XAVANTINA,3649617,1074892,-14.672500,-52.355000,Active,RIO DAS MORTES,266.00,"24950,00 km²",26100000,BR - BRAZIL,South America,Yes,Yes,0.786505,0.977071,3649618,24950.00,123
807,"YADKIN COLLEGE, NC",4148321,1069484,35.856700,-80.386900,Active,YADKIN RIVER,194.60,"5905,00 km²",02116500,US - UNITED STATES,North America,Yes,Yes,0.718625,0.896280,4148320,5905.00,294


In [7]:
wyield_folder = r"Z:\PhD_Datasets&Analysis\Outputs\T&M_WBM\wyield2"

years = range(1958, 2023 + 1)

# Read the zonal statistics files for each year and concatenate them into a single DataFrame
zonal_files = []
for year in years:
    zonal_files.extend(glob.glob(wyield_folder + f"\\wyield_zonal_statistics_{year}.csv"))
zonal_files.sort()
zonal_df = pd.concat([pd.read_csv(file) for file in zonal_files], ignore_index=True)
zonal_df = zonal_df.rename(columns={"grdcno_int": "station_no"})
zonal_df = zonal_df.set_index("station_no")
zonal_df["DATE"] = zonal_df["YEAR"].astype(str) + "-" + zonal_df["MONTH"].astype(str).str.zfill(2)
zonal_df["DATE"] = pd.to_datetime(zonal_df["DATE"], format="%Y-%m").dt.strftime("%Y-%m")
zonal_df = zonal_df[["YEAR", "MONTH", "DATE", "COUNT", "MEAN"]]

zonal_df

,YEAR,MONTH,DATE,COUNT,MEAN
station_no,,,,,
3617110,1958,1,1958-01,29626.0,140.825340
3617110,1958,2,1958-02,29555.0,100.151988
3617110,1958,3,1958-03,29509.0,112.716666
3617110,1958,4,1958-04,29483.0,49.679580
3617110,1958,5,1958-05,29476.0,20.027374
...,...,...,...,...,...
6870640,2023,8,2023-08,222.0,5.122293
6870640,2023,9,2023-09,222.0,70.731393
6870640,2023,10,2023-10,222.0,30.939898


In [8]:
def calculate_streamflow(year, month, wyield, area_km2):
    num_days = calendar.monthrange(int(year), int(month))[1]
    streamflow = (wyield * area_km2 * 1000) / (num_days * 24 *60 *60)  # Convert to m3/s
    return streamflow

In [15]:
# Ensure the required columns exist in filtered_css
css_stations = filtered_css.loc[filtered_css["has_monthly_k"] == "Yes"].set_index("station_no") # Filter for stations with monthly k recession

# Create a dictionary mapping station_no to CATCHMENT_SIZE2
catchment_size_dict = css_stations["CATCHMENT_SIZE2"].to_dict()

# Calculate FLOW_CMS
zonal_df["FLOW_CMS"] = zonal_df.apply(
    lambda row: calculate_streamflow(row["YEAR"], row["MONTH"], row["MEAN"], catchment_size_dict[row.name]),
    axis=1
)

zonal_df.to_csv(wyield_folder + f"\\wyield2_zonal_statistics_{years[0]}-{years[-1]}.csv")
zonal_df

,YEAR,MONTH,DATE,COUNT,MEAN,FLOW_CMS
station_no,,,,,,
3617110,1958,1,1958-01,29626.0,140.825340,32020.098677
3617110,1958,2,1958-02,29555.0,100.151988,25211.871952
3617110,1958,3,1958-03,29509.0,112.716666,25628.901445
3617110,1958,4,1958-04,29483.0,49.679580,11672.401251
3617110,1958,5,1958-05,29476.0,20.027374,4553.715148
...,...,...,...,...,...,...
6870640,2023,8,2023-08,222.0,5.122293,3.798116
6870640,2023,9,2023-09,222.0,70.731393,54.194655
6870640,2023,10,2023-10,222.0,30.939898,22.941547


In [16]:
zonal_df.describe()

,YEAR,MONTH,COUNT,MEAN,FLOW_CMS
count,639913.000000,639913.000000,639913.000000,639913.000000,639913.000000
mean,1990.500854,6.499987,2584.413355,41.832343,1184.412669
std,19.050162,3.452075,13504.697330,71.122481,10226.605296
min,1958.000000,1.000000,2.000000,0.000000,0.000000
25%,1974.000000,3.000000,44.000000,3.959797,2.598842
50%,1991.000000,6.000000,191.000000,10.802176,13.782974
75%,2007.000000,9.000000,698.000000,50.419498,79.154632
max,2023.000000,12.000000,219376.000000,1759.140253,408811.227270


In [17]:
zonal_df[zonal_df["COUNT"] == 1] # Check for calculations with only one pixel

,YEAR,MONTH,DATE,COUNT,MEAN,FLOW_CMS
station_no,,,,,,


In [18]:
# Paths to input datasets
root_folder = r"Z:\PhD_Datasets&Analysis\Info_Inputs"
drain_areas_folder = root_folder + "\\Streamflow_Sts_Drainage_Areas"

# Read the shapefile using DBF
drain_areas_tb = DBF(drain_areas_folder + "\\GRDC_Watersheds\\CSS-WATERSHEDS-MERGE_FINAL_SELECTION.dbf")
# Convert to DataFrame
drain_areas_df = pd.DataFrame(iter(drain_areas_tb)).set_index("grdc_no")
drain_areas_df.index = drain_areas_df.index.astype(int)  # Convert index to int
drain_areas_df

,river,station,area,altitude,lat_org,long_org,lat_pp,long_pp,dist_km,area_calc,...,grdcno_int,GRDCCOUNTR,Continent,has_monthl,has_daily_,monthly_k_,daily_k_re,Next_Downs,CATCHMENT_,Priority
grdc_no,,,,,,,,,,,,,,,,,,,,,
3617110,RIO MAMORE,GUAJARA-MIRIM,609000.0,109.29,-10.7925,-65.3478,-10.7938,-65.3479,0.1,612073.0,...,3617110,BR - BRAZIL,South America,Yes,Yes,0.736213,0.982211,3627041,609000.0,17
3617811,RIO GUAPORE,PRINCIPE DA BEIRA,341000.0,-999.00,-12.4267,-64.4253,-12.4271,-64.4271,0.2,341647.4,...,3617811,BR - BRAZIL,South America,Yes,Yes,0.733124,0.984667,3627110,341000.0,21
3617812,RIO GUAPORE,PIMENTEIRAS,54200.0,-999.00,-13.4858,-61.0500,-13.4854,-61.0479,0.2,55743.9,...,3617812,BR - BRAZIL,South America,Yes,Yes,0.789579,0.985912,3627810,54200.0,74
3617814,RIO GUAPORE,PONTES E LACERDA,2990.0,230.00,-15.2153,-59.3539,-15.2104,-59.3604,0.9,3020.5,...,3617814,BR - BRAZIL,South America,Yes,Yes,0.840667,0.959591,3627813,2990.0,412
3618051,RIO NEGRO,SERRINHA,279945.0,-999.00,-0.4817,-64.8272,-0.4938,-64.8146,1.9,292404.2,...,3618051,BR - BRAZIL,South America,Yes,Yes,0.826092,0.977063,3618000,279945.0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6854601,IIJOKI,SUOLIJARVI -OUTLET,1313.0,150.00,65.1446,28.0690,65.1438,28.0646,0.2,1290.9,...,6854601,FI - FINLAND,Europe,Yes,Yes,0.402936,0.913624,6854600,1313.0,531
6854713,KEMIJOKI,OUNASJARVI - OUTLET,363.0,290.00,68.3961,23.7524,68.3960,23.7430,0.4,364.6,...,6854713,FI - FINLAND,Europe,Yes,Yes,0.571777,0.919590,6854712,363.0,659
6855411,VUOKSI,KAJOONJARVI - OUTLET,125.0,167.00,63.1519,28.8992,63.1440,28.9050,0.9,126.5,...,6855411,FI - FINLAND,Europe,Yes,Yes,0.482329,0.945775,6855409,125.0,762


In [19]:
drain_areas_df.describe()

,area,altitude,lat_org,long_org,lat_pp,long_pp,dist_km,area_calc,grdcno_int,monthly_k_,daily_k_re,CATCHMENT_,Priority
count,8.090000e+02,809.000000,809.000000,809.000000,809.000000,809.000000,809.000000,8.090000e+02,8.090000e+02,809.000000,809.000000,8.090000e+02,809.000000
mean,4.829893e+04,132.793808,32.309265,-51.277276,32.309562,-51.277199,0.325216,4.838642e+04,4.646741e+06,0.580957,0.852138,4.829893e+04,405.000000
std,2.657074e+05,567.553806,26.122792,68.760614,26.123586,68.760781,0.405662,2.661802e+05,9.559717e+05,0.131494,0.174608,2.657074e+05,233.682477
min,4.900000e+01,-999.000000,-43.140600,-162.882900,-43.139600,-162.868800,0.000000,4.850000e+01,3.617110e+06,0.000000,0.000000,4.900000e+01,1.000000
25%,6.500000e+02,37.440000,31.221300,-96.205200,31.222900,-96.206200,0.200000,6.529000e+02,4.119286e+06,0.500805,0.836372,6.500000e+02,203.000000
50%,3.167600e+03,184.750000,42.584600,-75.803000,42.585400,-75.802100,0.200000,3.171200e+03,4.149122e+06,0.593689,0.901677,3.167600e+03,405.000000
75%,1.135460e+04,353.860000,47.415000,0.580600,47.418800,0.581200,0.300000,1.136720e+04,5.304100e+06,0.667038,0.942007,1.135460e+04,607.000000
max,4.680000e+06,2380.610000,68.850200,153.158700,68.860400,153.160400,7.000000,4.671462e+06,6.870640e+06,0.958535,0.996740,4.680000e+06,809.000000
